# 使用vitis hls设计矩阵乘法器
# 01: Python运行矩阵乘法
> 该实验改编自Xilinx的ug871的矩阵乘法的例程，使用Xilinx Vitis HLS生成的矩阵乘法 IP 来进行3*3矩阵乘法操作


In [1]:
import numpy as np
import time
start_time = time.time()
a = np.array([[-1,1,1],[0,1,-1],[1,1,0]])
b = np.array([[1,1,-1],[0,1,1],[0,0,1]])
py_result = a@b
print(py_result)
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

[[-1  0  3]
 [ 0  1  0]
 [ 1  2  0]]
耗时：0.004723548889160156s


# 02：导入硬件，进行验证

In [2]:
from pynq import Overlay
overlay = Overlay("./matrix_mul.bit")
mat = overlay.matrix_mul_0

### 分配内存供IP使用
`pynq.allocate`函数用于为PL中的IP分配可以使用的内存空间。
- 在PL中的IP访问DRAM之前，必须为其保留一些内存供IP使用，分配大小与地址
- 我们分别为输入、输出分配内存，数据类型为int8
- `pynq.allocate`会分配物理上的连续内存，并返回一个`pynq.Buffer`表示已经分配缓冲区的对象

In [3]:
from pynq import allocate
a_buffer = allocate(shape=(3,3,), dtype='i1')
b_buffer = allocate(shape=(3,3,), dtype='i1')
res_buffer = allocate(shape=(3,3,), dtype='i2')

In [4]:
np.copyto(a_buffer, a)
np.copyto(b_buffer, b)
np.copyto(res_buffer, np.array([[0,0,0],[0,0,0],[0,0,0]]))
print(a_buffer)
print(b_buffer)
print(res_buffer)

[[-1  1  1]
 [ 0  1 -1]
 [ 1  1  0]]
[[ 1  1 -1]
 [ 0  1  1]
 [ 0  0  1]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]


### 配置IP

我们可以直接使用IP的`write`方法，将刚分配的内存空间的地址写入到IP对应位置上

In [5]:
mat.s_axi_control.write(0x10,a_buffer.physical_address)
mat.s_axi_control.write(0x1c,b_buffer.physical_address)
mat.s_axi_control.write(0x28,res_buffer.physical_address)

### 启动IP

控制信号位于0x00地址，我们可以对其进行写入与读取来控制IP启动、监听是否完成。

In [6]:
import time

mat.s_axi_CTRL.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = mat.s_axi_CTRL.read(0x00)
    if reg != 1:
        break
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0010275840759277344s


结果已经被写入到了`res_buffer`中，我们可以进行查看

In [7]:
print(py_result)
print(res_buffer)

[[-1  0  3]
 [ 0  1  0]
 [ 1  2  0]]
[[-1  0  3]
 [ 0  1  0]
 [ 1  2  0]]
